In [0]:
import mlflow
from mlflow import pyfunc
import pandas as pd
import numpy as np
from datetime import datetime
from mlflow.tracking import MlflowClient
'''
Filter and feature generator
'''
def feature_mom_gen(file,last_date,filter_mon,grp_cols,date,agg_cols,renam_cols):
    past_date = datetime.strptime(last_date,'%Y-%m-%d') -pd.DateOffset(months=filter_mon)
    file[date]=pd.to_datetime(file[date],dayfirst='true')
    tmp=file[np.logical_and(file[date]<=last_date,file[date]>=past_date)]
    t=tmp.groupby(grp_cols,as_index=False).agg(agg_cols)
    t.columns=renam_cols
    return(t)

'''
Column name standizer
'''
def name_standard(frame,id,columns):
    c=frame.columns[~frame.columns.isin(columns)]
    c=c[~c.isin(id)]
    df=pd.DataFrame()
    p=[]
    for k in c:
        frame1 = pd.pivot_table(frame, values =k, index =id,columns=columns,
                              aggfunc = np.sum)
        cols=frame1.columns
        a=[]
        for i in np.arange(len(cols)):
            
            tmp=''
            for j in np.arange(len(cols[i])):
                tmp+=cols[i][j]+'_'
            tmp=k+'_'+tmp
            a.append(tmp)
        
        b=['enq_cust']
        b.extend(a)
        df=frame1.reset_index()
        df.columns=b
        p.append(df)
    df=pd.DataFrame()
    for i in p:
        if len(df)==0:
            df=i
        else:
            df=df.merge(i,how='left',left_on=['enq_cust'],right_on=['enq_cust'])
  
    return df
    

In [0]:
'''
Enq Rollup
'''
from databricks.feature_store import feature_table
@feature_table
def enq_rllup(file,last_date):
    '''
    Enq Inst level rollup
    '''
    feature_base=((feature_mom_gen(file,last_date,20,['en_inst','enq_class','enq_cust'],'date',{'amount':['sum','count']},['en_inst','enq_class','enq_cust','sum_amt_20m','cnt_20m']).merge(
                   feature_mom_gen(file,last_date,9,['en_inst','enq_class','enq_cust'],'date',{'amount':['sum','count']},['en_inst','enq_class','enq_cust','sum_amt_9m','cnt_9m']),how='left')).merge(
        feature_mom_gen(file,last_date,6,['en_inst','enq_class','enq_cust'],'date',{'amount':['sum','count']},['en_inst','enq_class','enq_cust','sum_amt_6m','cnt_6m']),how='left')).merge(
        feature_mom_gen(file,last_date,3,['en_inst','enq_class','enq_cust'],'date',{'amount':['sum','count']},['en_inst','enq_class','enq_cust','sum_amt_3m','cnt_3m']),how='left')

    feature_base=name_standard(feature_base,['enq_cust'],['en_inst','enq_class'])
    feature_base=feature_base.fillna(0)
    
    '''
    Enq Class level rollup
    '''
    feature_base1=((feature_mom_gen(file,last_date,20,['enq_class','enq_cust'],'date',{'amount':['sum','count']},['enq_class','enq_cust','sum_amt_250m','cnt_20m']).merge(
        feature_mom_gen(file,last_date,9,['enq_class','enq_cust'],'date',{'amount':['sum','count']},['enq_class','enq_cust','sum_amt_9m','cnt_9m']),how='left')).merge(
        feature_mom_gen(file,last_date,6,['enq_class','enq_cust'],'date',{'amount':['sum','count']},['enq_class','enq_cust','sum_amt_6m','cnt_6m']),how='left')).merge(
        feature_mom_gen(file,last_date,3,['enq_class','enq_cust'],'date',{'amount':['sum','count']},['enq_class','enq_cust','sum_amt_3m','cnt_3m']),how='left')
    feature_base1=name_standard(feature_base1,['enq_cust'],['enq_class'])
    feature_base1=feature_base1.fillna(0)
    
    
    '''
    Enq cust level rollup
    '''
    feature_base2=((feature_mom_gen(file,last_date,20,['enq_cust'],'date',{'amount':['sum','count']},['enq_cust','sum_amt_250m','cnt_20m']).merge(
        feature_mom_gen(file,last_date,9,['enq_cust'],'date',{'amount':['sum','count']},['enq_cust','sum_amt_9m','cnt_9m']),how='left')).merge(
        feature_mom_gen(file,last_date,6,['enq_cust'],'date',{'amount':['sum','count']},['enq_cust','sum_amt_6m','cnt_6m']),how='left')).merge(
        feature_mom_gen(file,last_date,3,['enq_cust'],'date',{'amount':['sum','count']},['enq_cust','sum_amt_3m','cnt_3m']),how='left')
    feature_base2=feature_base2.fillna(0)


    feature_base_final=(feature_base.merge(feature_base1,how='left',left_on=['enq_cust'],right_on=['enq_cust'])).merge(feature_base2,how='left',left_on=['enq_cust'],right_on=['enq_cust'])
    feature_base_final.fillna(0)
    
    feature_base_final=(feature_base.merge(feature_base1,how='left',left_on=['enq_cust'],right_on=['enq_cust'])).merge(feature_base2,how='left',left_on=['enq_cust'],right_on=['enq_cust'])
    feature_base_final.fillna(0)
    
    
    for i in feature_base_final.columns:
        if i!='enq_cust':
            feature_base_final[i]=feature_base_final[i].astype('float64')

    return(spark.createDataFrame(feature_base_final))





In [0]:
'''
Latest version of model
'''
def get_latest_model_version(model_name):
  latest_version = 1
  mlflow_client = MlflowClient()
  for mv in mlflow_client.search_model_versions(f"name='{model_name}'"):
    version_int = int(mv.version)
    if version_int > latest_version:
      latest_version = version_int
  #latest_model_version = get_latest_model_version(model_name)
  model_uri = f"models:/enq_sample_model/{latest_version}"
  return model_uri
